# Solving Garden Optimization Problem with the QAOA algorithm

![Garden](planting.jpg)

In this tutorial, we will demonstrate how to generate an instance of the Garden Optimization Problem [[1]](#ref) and solve it using the Quantum Approximate Optimization Algorithm (QAOA) [[2]](#ref).

To achieve this, we will be utilizing [Qiskit](https://qiskit.org/), a quantum development toolkit that offers a variety of implementations of common quantum optimization techniques including QAOA. Our primary objective is to gain a fundamental understanding of QAOA's optimization algorithm and how it can be utilized to solve combinatorial optimization problems.

For additional technical details and examples, we recommend referring to the Qiskit documentation and the optimization tutorials page [[3]](#ref).

## Table of Contents

* [Garden Optimization Problem](#qubo)
* [Classical solver](#exact)
* [QAOA optimization](#qaoa)
* [References](#ref)

## Relevant packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

# import sys, os
# sys.path.append(os.path.abspath("<full path to GardenOptimizationProblem.py>"))
from GardenOptimizationProblem import GardenOptimizationProblem
from sampling import sample_most_likely

from qiskit import Aer, execute
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms import NumPyMinimumEigensolver

%matplotlib inline

## <a class="anchor" id="qubo"> </a>Garden Optimization Problem

The goal of the Garden Optimization Problem (GOP) [[1]](#ref) is to find an optimal placement of `n` plants into `n` garden pots (one plant per pot). Each plant belongs to a particular species, and in fact, some species like to be placed next to each other, while others do not. An optimal placement of the `n` plants is thus a placement that maximizes the number of friendly relationships between neighboring pots.

In addition, the following constraints are added to influence the optimal plant placement:

1. "*Fill all pots*": Each of the `n` pots must be filled with exactly one plant.
2. "*Place all plants*": All of the `n` plants are placed in the garden.
3. "*Always look on the bright side of life*": (additional level of complexity) Large plants should not overshadow smaller plants.

Workshop participants are welcome to learn more details about GOP and follow all the steps necessary to complete the binary problem formulation as provided in this [Tutorial](https://jugit.fz-juelich.de/qip/springschool2023/-/blob/main/garden_DWAVE/garden_DWAVE.ipynb). 
For the purpose of this tutorial, we skip this step and already provide the final binary problem formulation in the class `GardenOptimizationProblem` on only concentrate on the actual model setup.
When choosing the problem parameters, please keep in mind the limitation on the maximum problem size (number of variables) that can be treated with quantum simulators. To be able to complete the tutorial with your local PC, the number of the variables (= the number of qubits) should not exceed 24.

The GOP setup consists of 2 steps: defining the arrangement of the plant pots and selecting the plant species that should find its best location in the pots. For simplicity, let us consider the simplest $2 \times 2$ arrangement of pots. Taking into account the 4 possible different types of plants that can be chosen to be placed in the pots, our optimization problem has a total of 16 variables.

In [ ]:
cols = 2
rows = 2

model = GardenOptimizationProblem()
model.build_garden(cols, rows, verbose=True)

The collection of plants we are going to cultivate is already preset in the `GardenOptimizationProblem` instance.
Let us randomly select 4 plants.

In [ ]:
species, count, size = model.pick_random_plants(replacement=False, seed=42)
model.get_plants(species, count, size, companions="./companions.csv", verbose=True)

The companion matrix tells us which plants a given species prefers to have as neighbors (-1: like, 0: neutral, 1: dislike).

The last step in the problem initialization is to formulate the Quadratic Unconstrained Binary Optimization (QUBO) problem. The QUBO formulation of GOP contains 3 parameters (Lagrangian coefficients due to the chosen constraints).

In [ ]:
# Lagrangian parameters (lambda_1, lambda_2, lambda_3)
lagrangian_params = (2, 2, 1)

# output is the DOCplex binary optimization model
mdl = model.build_qubo(*lagrangian_params, verbose=False)

For convenience, we store the QUBO problem in the [DOcplex model](https://www.ibm.com/docs/en/icos/12.9.0?topic=docplex-python-modeling-api) format. This format is widely used to execute the IBM Decision Optimization CPLEX optimizer from Python and has an interface to Qiskit.

In [ ]:
mdl.prettyprint()

## <a class="anchor" id="exact"> </a>Classical solver

The chosen problem size allows us to quickly estimate the solution with the simplest "brute-force" search. 

<font size="4"><span style='background:yellow'>**Exercise:**</span></font> Implement the brute-force search for the Garden Optimization Problem given the problem objective/cost function can be computed with the `model.evaluate_objective(x)` method, where `x` is a solution candidate (specified as a list of binary bits). How many solutions do you get? How does the execution time scales with the problem size?

Hint: The "brute-force" search essentially means to check the energies of all possible problem solutions and select the lowest ones. Use the Numpy routine `np.binary_repr()` to obtain the binary strings representing solutions. Additionally, to match the problem binary encoding, the strings should be reverted. 

In [ ]:
# insert your code here

In [ ]:
# visualize the brute-force search solution
# model.plot_solution(x_bf)

Obviously, brute-force search quickly becomes intractable due to the exponential growth of the configuration space. Therefore, for larger problems, one must use an efficient classical solver, e.g., this can be done very easily with the `solve()` method of our DOcplex model, provided that IBM's CPLEX solver is available.

In [ ]:
#----------------------------------------------------------------
# Warning: The CPLEX solver is not installed on the JUNIQ cloud!
#
# The code below would work on you local PC if you followed 
# the installation instructions mentioned in README
#----------------------------------------------------------------

# mdl.solve()
# mdl.print_solution()

# x = dict()
# for v in mdl.iter_binary_vars():
#     x[v.index] = v.solution_value

# # Visualize the classical solver solution
# model.plot_solution(x)

## <a class="anchor" id="qaoa"> </a>QAOA optimization

QAOA is a popular hybrid quantum-classical variational algorithm that is designed for solving the combinatorial optimization problems on NISQ devices. Here is the QAOA algorithm flowchart that we are going to implement below using Qiskit facilities.

![QAOA](qaoa.jpg)

### From QUBO to Ising model

To setup the QAOA quantum algorithm, we need to convert the binary problem into the corresponding Ising Hamiltonian, which is further transformed into a quantum circuit. Qiskit automates this conversion:

In [ ]:
from qiskit_optimization.translators import from_docplex_mp

qp = from_docplex_mp(mdl)
qubitOp, offset = qp.to_ising()

print("Offset:", offset)
print("Ising Hamiltonian:\n", str(qubitOp))

`qubitOp` contains representation of the objective function in terms of the Pauli's `Z` and `ZZ` operators. `offset` is a constant (physically irrelevant energy shift) that is basically required to obtain the absolute value of the objective function for comparison with classical methods.

### QAOA ansatz

The next step is to construct the QAOA variational ansatz. This can be done explicitly using the basic Qiskit quantum circuit constructor routines. Instead, for simplicity, we will use the generic constructor already provided by Qiskit. The key parameter is the depth of the ansatz `p`.

In [ ]:
from qiskit.circuit.library.n_local.qaoa_ansatz import QAOAAnsatz

p = 1

qaoa_ansatz = QAOAAnsatz(qubitOp, reps=p)

print(qaoa_ansatz.draw())
# print(qaoa_ansatz.decompose().draw())

If you want to see the explicit circuit structure, uncomment the bottom line. But beware of the output verbosity!

From a practical point of view, one may also be interested in knowing the properties of the quantum circuit, such as the circuit depth and the number of gates involved. This can be done with

In [ ]:
from qiskit.compiler import transpile
# from qiskit.providers.fake_provider import FakeGuadalupeV2

backend = Aer.get_backend("qasm_simulator")
# backend = FakeGuadalupeV2()

ansatz = transpile(qaoa_ansatz, backend)

print("QAOA p:", p)
print("Number of qubits:", ansatz.num_qubits)
print("Circuit depth:", ansatz.depth())
print("Number of gates:", list(ansatz.count_ops().items()))

Thus, by choosing an appropriate `backend`, one could investigate the circuit structure requirements for a real quantum hardware.

<font size="4"><span style='background:yellow'>**Exercise:**</span></font> 
Discuss the resources that would be required to execute the QAOA algorithm on a "fake" quantum device.
The fake backends are built to mimic the behaviors of IBM Quantum devices and contain important information about the quantum system such as coupling map, basis gates, qubit properties (T1, T2, error rate, etc.) which are useful for testing the transpiler and performing noisy simulation of the system.

### Exact diagonalization

As for the classical solvers, in the case of small system sizes, the Ising Hamiltonian can be diagonalized using the classical approach, for instance, involving routines from NumPy.

In [ ]:
# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver()
result = ee.compute_minimum_eigenvalue(qubitOp)

print("Minimized objective:", result.eigenvalue.real + offset)

x_exact = sample_most_likely(result.eigenstate)
obj_exact = qp.objective.evaluate(x_exact)
print("Solution:", x_exact)
print("Solution objective:", obj_exact)

model.plot_solution(x_exact)

Here you have the opportunity to compare the obtained solutions with the ones you might have computed earlier with the "brute-force" method.

### Quantum computer

Quantum computing offers an alternative algorithm to estimate the lowest eigenstates of the Ising Hamiltonian - the quantum variational eigensolver with the QAOA variational ansatz. Besides the variational ansatz, the algorithm requires the specification of a classical local optimizer (`opt=COBYLA()`), which is used to optimize the circuit parameters $\beta$ and $\gamma$. The QAOA solutions are known to be extremely sensitive to the choice of the starting point. To speed up the optimization procedure, we provide a "good" starting point for the tutorial problem that quickly leads to a correct solution.

The Qiskit's Aer provider contains a variety of high performance simulator backends for a variety of simulation methods. The available backends on the current system can be viewed using `Aer.backends`. We choose the shot-based quantum computer simulator and set the seed value of the random number generator for reproducibility.

In [ ]:
seed = 12345
algorithm_globals.random_seed = seed

backend = Aer.get_backend("qasm_simulator")

n_shots = 129
quantum_instance = QuantumInstance(
    backend, seed_simulator=seed, seed_transpiler=seed, shots=n_shots
)

In [ ]:
%%time

p = 1

# to visualize the optimization history
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean+offset)
    
initial_point = [0.5, -0.25]
    
opt = COBYLA(maxiter=100)
qaoa = QAOA(
    optimizer=opt, reps=p, 
    quantum_instance=quantum_instance, 
    initial_point=initial_point, 
    callback=store_intermediate_result,
)

out = qaoa.compute_minimum_eigenvalue(qubitOp)
opt_params = list(out.optimal_parameters.values())

print()
print("Expectation value: {0:.4f}".format(out.eigenvalue.real + offset))
print("Optimal parameters (β, γ): {0}".format(opt_params))
print()

In [ ]:
plt.figure(figsize=(6, 5))
plt.plot(counts, values, label=type(opt).__name__)
plt.xlabel("Number of function evaluations")
plt.ylabel("Expectation value")
plt.title("Objective function convergence")
plt.legend()
plt.show()

Let us discuss the workflow and output of the QAOA optimization procedure. When using the shot-based simulator, the QAOA algorithm generates `n_shots` binary strings, each of which could be considered as a candidate solution. 
On the classical backend, the objective values are computed for each string. The expectation value (mean objective) is used to optimize the variational parameters following the instructions of the local classical optimization algorithm. The quantum/classical optimization procedure terminates after satisfying the stopping criteria (`maxiter`).

The main output of the QAOA optimization procedure is a list of variational parameters/angles. We use these angles to setup the optimized QAOA variational ansatz and sample the final state from it.

In [ ]:
def sampling(
    ansatz, params, n_shots=128, backend=Aer.get_backend("qasm_simulator"), seed=12345
):
    qc = ansatz.copy()
    qc.measure_all()

    job = execute(
        qc.assign_parameters(parameters=params),
        backend,
        shots=n_shots,
        seed_simulator=seed,
        seed_transpiler=seed,
    )
    counts = job.result().get_counts()

    sols = []
    for s, p in counts.items():
        x = np.asarray([int(y) for y in reversed(list(s))])
        obj = qp.objective.evaluate(x)
        sols.append({"x": x, "prob": p / n_shots, "obj": obj, "feas": model.is_feasible(x)})

    return sols


def plot_solutions(dict_sols, exact, show_feasible=False, sort_obj=True):
    """Visualize the QAOA solutions.
    
    Args:
        dict_sols: dictionary {"label": solutions_dict, ...} of data for visualization
        exact: reference value for the objective function
        show_feasible: highlight the feasible solutions
        sort_obj: sort the objective function values
    """
    fig, ax = plt.subplots(1, 2, figsize=(11, 5))

    for label, sol in dict_sols.items():
        if sort_obj:
            sol = sorted(sol, key=lambda d: d["obj"])
        
        x = [s["obj"] for s in sol]
        y = [s["prob"] for s in sol]
        z = [s["feas"] for s in sol]

        ax[0].plot(x)
        ax[0].set_xlabel("Samples")
        ax[0].set_ylabel("Objective")

        ax[1].bar(x, y, width=0.5, label=label)
        if show_feasible:
            x_feas, y_feas = [], []
            for _x, _y, _z in zip(x, y, z):
                if _z:
                    x_feas.append(_x)
                    y_feas.append(_y)
            ax[1].bar(x_feas, y_feas, width=1.0, label=label + " (feas)", color="green", fill=False, lw=3, edgecolor="green")
        ax[1].set_xlabel("Objective")
        ax[1].set_ylabel("Quasi probability")
    
    # exact solution
    ax[0].axhline(y=exact, ls=":", color='k')
    ax[1].axvline(x=exact, ls=":", color='k', label='exact')

    plt.legend()
    plt.show()

In [ ]:
sols = sampling(
    qaoa.ansatz, out.optimal_parameters, backend=backend, n_shots=128, seed=12345
)

plot_solutions({"QAOA": sols}, obj_exact, show_feasible=True)

On the left, we plot the objectives for all output samples. The black dotted line shows the exact value (taken from the exact solver). On the right, we see the quasi probability to find the solution at a given energy. Unlike the exact solver, the QAOA returns multiple solutions in a wide energy interval. With the green bars we mark the "feasible" solutions, i.e. the solution that satisfy all the constraints.

Let us take a closer look at the solutions with the lowest objectives.

In [ ]:
print("Lowest objective solutions:")
lowest_obj_solutions = sorted(sols, key=lambda kv: kv["obj"])
for s in lowest_obj_solutions[:8]:
    print(s)

The string with the lowest objective value has the same energy as the one obtained earlier with the exact diagonalization. We might notice the exact and QAOA strings differ but having the same objective - the ground state solution is degenerate.

In [ ]:
model.plot_solution(lowest_obj_solutions[0]["x"])

As it follows from the plot, the next solution has a higher objective but also a higher sampling probability. Let us examine this solution.

In [ ]:
model.plot_solution(lowest_obj_solutions[1]["x"])

This sample does not satisfy all the applied problem constraints. Nevertheless, this sample could still be considered as a near optimal solution.

### Energy surface

For `p=1`, one can visualize the cost energy surface as a function of the $\beta$ and $\gamma$ angles.

In [ ]:
%%time

N1 = 20
N2 = 10
beta = np.linspace(start=-np.pi, stop=np.pi, num=N1)
gamma = np.linspace(start=-np.pi/2, stop=np.pi/2, num=N2)

e = qaoa.get_energy_evaluation(qubitOp, return_expectation=False)

energy = np.zeros((N1, N2))
for i, b in enumerate(beta):
    for j, g in enumerate(gamma):
        params = [b, g]
        energy[i, j] = e(params) + offset

We can plot the energy surface as a heatmap and add the location (cross) of the optimal angles found from the QAOA run above. 

In [ ]:
fig = plt.figure(figsize=(6, 6), dpi=80, facecolor="w", edgecolor="k")

X, Y = np.meshgrid(beta, gamma)

ax = plt.gca()
im = ax.imshow(
    energy.T,
    interpolation=None,
    origin="lower",
    extent=[X.min(), X.max(), Y.min(), Y.max()],
    cmap=cm.coolwarm,
    vmax=None,
    aspect="auto",
)
plt.title("Expectation value")
plt.xlabel(r"$\beta$")
plt.ylabel(r"$\gamma$")

ax.scatter(opt_params[0], opt_params[1], marker="X", s=500, c="black")

from mpl_toolkits.axes_grid1 import make_axes_locatable

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.show()

The plot clearly shows us the regions to initialize the optimization parameters in the case of p=1. Note the characteristics of the visualized pattern. Due to the shape of the mixing operator, we can clearly see the periodicity in $\beta$. However, there is no periodicity along $\gamma$. We should always keep this in mind when searching for the optimal parameters.

<font size="4"><span style='background:yellow'>**Exercise:**</span></font> Try to visualize the energy surface in 3D.

In [ ]:
# insert your code here

<font size="4"><span style='background:yellow'>**Exercise:**</span></font> Efficient parameter initialization in QAOA is an active area of research. We offer you to check the QAOA solutions computed with random initializations. Moreover, you could also play with other [local optimizers](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.html) available in Qiskit.

<font size="4"><span style='background:yellow'>**(optional) Exercise:**</span></font> We use the shot-based quantum simulator such that the final state is estimated statistically by setting the number of shots (`n_shots`).
Therefore, it is also interesting to check the influence of `n_shots` on the output of the QAOA algorithm.

In [ ]:
# insert your code here

<font size="4"><span style='background:yellow'>**Exercise:**</span></font> Above we were trying to find problem solutions with the simplest `p=1` QAOA ansatz. Could you examine what happpens when you increase `p`?

In [ ]:
# insert your code here

Increasing the QAOA circuit depth has an important effect that helps to improve the solutions. As we could see, the solver concentrates the samples in the low energy region. As a result, we might expect to incease the probability to find an optimal solution. Note, however, that the initialization of the QAOA parameters was not random! The success of the QAOA search would still be strongly influenced by the initialization strategy [[4]](#ref).

## Other topics discussed in this course

In this notebook, we have covered how to solve the Garden Optimization Problem using the Quantum Approximate Optimization Algorithm (QAOA). We used the Qiskit quantum developement toolkit to setup the quantum variational ansatz and the computational workflow to emulate the quantum computer.

To go deeper into a topic how to formulate the Garden Optimization Problem and encode it into an Quadratic Unconstrained Binary Optimization (QUBO) problem, please check the tutorial ["Garden Optimization Problems"](https://jugit.fz-juelich.de/qip/springschool2023/-/blob/main/garden_DWAVE/garden_DWAVE.ipynb). In that tutorial you will also learn how to find solutions using an alternative quantum optimization algorithm and how to execute it on the D-Wave Quantum Annealer.

Another example of a "real-world" optimization problem is presented in the ["Knapsack problem"](https://jugit.fz-juelich.de/qip/springschool2023/-/blob/main/knapsack_QAOA/knapsack-qaoa.ipynb) tutorial. 
There, you can further practice deriving the optimization problem in the QUBO formulation and using the Qiskit quantum development toolkit to implement the QAOA algorithm.

Another very famous example of the real and challenging optimization problem is studied in the tutorial ["Travelling Salesperson Problem"](https://jugit.fz-juelich.de/qip/springschool2023/-/tree/main/tsp_DWAVE). This tutorial will introduce you to the exciting world of the Quantum Annealers.

## <a class="anchor" id="ref"> </a>References

[1] Gonzalez Calaza, C. D.; Willsch, D.; Michielsen, K. Garden Optimization Problems for Benchmarking Quantum Annealers. Quantum Inf Process 2021, 20 (9), 305. https://doi.org/10.1007/s11128-021-03226-6.

[2] Edward Farhi, Jeffrey Goldstone, and Sam Gutmann. "A quantum approximate optimization algorithm." [arXiv preprint arXiv:1411.4028 (2014)](https://arxiv.org/abs/1411.4028).

[3] https://qiskit.org/documentation/optimization/

[4] Zhou, L. et al., Quantum Approximate Optimization Algorithm: Performance, Mechanism, and Implementation on Near-Term Devices. Phys. Rev. X 2020, 10 (2), 021067. https://doi.org/10.1103/PhysRevX.10.021067.

In [ ]:
import qiskit.tools.jupyter

%qiskit_version_table